In [1]:
import pandas as pd
import numpy as np

In [2]:
target = 'popularity'
origin_df = pd.read_csv('../Data/SpotifyFeatures.csv')
df = origin_df.copy()

In [3]:
top_100s = pd.read_csv('../Data/songs_normalize.csv')
top_100s.head()

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop


In [4]:
cutline = top_100s.popularity.mean()
df[target] = df[target]>=cutline
df[target] = df[target].replace([False, True], [0, 1])

In [5]:
df['genre'] = df['genre'].replace("Children’s Music", "Children's Music")

In [6]:
df['duration_ms'] = round(df['duration_ms'] / 1000 / 60, 2)
df.rename(columns={'duration_ms' : 'duration_m'}, inplace=True)

In [7]:
delete = ['artist_name', 'track_name', 'track_id']
df = df.drop(delete, axis=1)

In [8]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
long_dur = train.query("duration_m>7").index
train = train.drop(long_dur)

In [11]:
delete = ['Anime', 'Comedy', 'Opera', 'Movie', 'A Capella', 'Classical', "Children's Music"]
train = train[~train.genre.isin(delete)]

In [12]:
features = df.drop(target, axis=1).columns

In [13]:
def x_y_split(df) :
    X = df[features]
    y = df[target]
    return X, y

In [14]:
X_train, y_train = x_y_split(train)
X_test, y_test = x_y_split(test)

In [15]:
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from xgboost import XGBClassifier

In [16]:
xgb_pipe = make_pipeline(
            OrdinalEncoder(),
            XGBClassifier(
                objective="binary:logistic",
                eval_metric="error",
                random_state=42,
                n_jobs=-1,
                colsample_bytree=0.7000000000000001,
                learning_rate=0.04,
                max_depth=7,
                min_child_weight=11,
                n_estimators=2500,
                reg_alpha=0,
                reg_lambda=6,
                scale_pos_weight=2.704585271920224,
            ))

In [17]:
xgb_pipe.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['genre', 'key', 'mode', 'time_signature'],
                                mapping=[{'col': 'genre',
                                          'data_type': dtype('O'),
                                          'mapping': Rap             1
Dance           2
Country         3
Electronic      4
Blues           5
R&B             6
Pop             7
Hip-Hop         8
Folk            9
Soundtrack     10
Ska            11
Reggae         12
Alternative    13
Jazz           14
Soul           15
World          16
Indie          17
Reggaeton      18
Rock           19
NaN            -2
dtype: int64},
                                         {'col': 'key', 'data_type': dtype('O'),
                                          'mapping...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.04, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=7, max_leaves=0, min_child_weight=11,
                               missing=nan, monotone_constraints='()',
                               n_estimators=2500, n_jobs=-1,
                               num_parallel_tree=1, predictor='auto',
                               random_state=42, reg_alpha=0, reg_lambda=6, ...))])

In [18]:
X_test

,genre,acousticness,danceability,duration_m,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
151557,Pop,0.09490,0.758,2.75,0.944,0.000006,D,0.4020,-1.541,Minor,0.0696,119.970,4/4,0.6960
29413,Anime,0.91500,0.512,4.83,0.259,0.000000,F,0.0853,-9.646,Major,0.0322,135.829,4/4,0.2790
226549,Soul,0.51700,0.510,2.77,0.666,0.013900,C,0.3390,-8.162,Minor,0.0920,167.134,4/4,0.7010
51886,Blues,0.81000,0.583,2.91,0.594,0.000000,G,0.1170,-6.373,Major,0.0571,124.306,4/4,0.8070
107213,Opera,0.94400,0.402,2.73,0.296,0.000015,F#,0.5920,-17.452,Major,0.1350,89.898,5/4,0.1940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148101,Jazz,0.48000,0.467,5.50,0.389,0.000000,A#,0.0835,-10.633,Minor,0.0305,103.379,4/4,0.4220
169392,Comedy,0.58500,0.450,1.57,0.989,0.000000,F,0.9380,-5.680,Major,0.9090,82.908,3/4,0.0391
121153,Rap,0.68900,0.638,4.41,0.787,0.000000,C#,0.1120,-6.093,Minor,0.1480,83.610,4/4,0.3120
138120,Reggae,0.02020,0.830,3.28,0.753,0.000000,C,0.1100,-6.276,Minor,0.1920,97.997,4/4,0.4650


In [19]:
X_train

,genre,acousticness,danceability,duration_m,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
115817,Rap,0.166000,0.567,5.35,0.309,0.000000,G#,0.1200,-14.254,Major,0.3850,65.041,4/4,0.509
15293,Dance,0.225000,0.677,6.51,0.852,0.000039,F,0.0595,-10.856,Major,0.0450,91.691,4/4,0.930
8959,Country,0.030900,0.551,3.73,0.909,0.000000,A,0.1840,-5.380,Major,0.0514,118.967,4/4,0.559
25858,Electronic,0.000259,0.504,3.49,0.938,0.007390,F,0.3060,-4.048,Minor,0.2010,75.032,4/4,0.217
70955,Blues,0.219000,0.655,3.05,0.520,0.000354,G,0.1090,-11.292,Major,0.0269,119.527,4/4,0.488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,Pop,0.006030,0.809,2.83,0.636,0.026400,C#,0.2740,-5.923,Minor,0.2170,102.012,4/4,0.569
119879,Rap,0.171000,0.595,2.59,0.533,0.014300,G#,0.0959,-8.430,Major,0.0407,109.984,4/4,0.145
131932,Reggae,0.027200,0.530,4.12,0.402,0.000281,G,0.9360,-8.917,Minor,0.0354,129.762,4/4,0.667
146867,Jazz,0.818000,0.598,6.12,0.291,0.000052,C#,0.1070,-10.748,Major,0.0412,115.351,4/4,0.360


In [22]:
X_test.iloc[0]

genre                    Pop
acousticness          0.0949
danceability           0.758
duration_m              2.75
energy                 0.944
instrumentalness    0.000006
key                        D
liveness               0.402
loudness              -1.541
mode                   Minor
speechiness           0.0696
tempo                 119.97
time_signature           4/4
valence                0.696
Name: 151557, dtype: object

In [21]:
# X_test = [['Pop', 0.5, 0.5, 5, 0.5, 0.5, 'C#', 0.5, -50, 'Major', 0.5, 100, '4/4', 0.5]]
test = [['Jazz', 0.5, 0.5, 5, 0.5, 0.5, 'C#', 0.5, -50, 'Major', 0.5, 100, '4/4', 0.5]], columns=X_train.columns
result = xgb_pipe.predict(test)
result

SyntaxError: cannot assign to literal (1021683219.py, line 2)